In [36]:
import sqlite3
import pandas as pd
from collections import defaultdict
import numpy as np

In [37]:
db_path = "/Users/jiatongchoo/Desktop/Recruit/Tesla/Project/vehicle_allocation.db"
con = sqlite3.connect(db_path)

In [ ]:
plants = pd.read_csv("plants.csv")
colors = pd.read_csv("colors.csv")
models = pd.read_csv("models.csv")
trims = pd.read_csv("trims.csv")
regions = pd.read_csv("regions.csv")
demand = pd.read_csv("regional_demand.csv")
vehicle_production = pd.read_csv("vehicle_production.csv")
transit_time = pd.read_csv("transit_time.csv")

In [39]:
vehicle_production["prod_date"] = pd.to_datetime(vehicle_production["prod_date"])
demand["week_start"] = pd.to_datetime(demand["week_start"])

In [ ]:
vehicle_production["week_start"] = vehicle_production["prod_date"] - pd.to_timedelta(
    vehicle_production["prod_date"].dt.dayofweek, unit="D"
)
base_supply = (
    vehicle_production.groupby(
        ["week_start", "plant_id", "model_code", "trim_code", "color_code"], as_index=False
    )["qty"].sum()
    .rename(columns={"qty": "supply_qty"})
)

In [41]:
weekly_demand = demand.rename(columns={"forecast_qty":"demand_qty"}).copy()

In [ ]:
base_demand = (
    demand.groupby(
        ["week_start", "region_id", "model_code", "trim_code", "color_code"], as_index=False
    )["forecast_qty"].sum()
    .rename(columns={"forecast_qty": "demand_qty"})
)

In [44]:
lead_matrix = transit_time.copy()

In [ ]:
def run_greedy_allocation(supply_df, demand_df, lead_df, scenario_id):
    allocations = []

    for week in sorted(demand_df["week_start"].unique()):
        week_supply = supply_df[supply_df["week_start"] == week].copy()
        week_demand = demand_df[demand_df["week_start"] == week].copy()

        for _, d_row in week_demand.iterrows():
            remaining = d_row["demand_qty"]

            candidates = lead_df.query("region_id == @d_row.region_id")[["plant_id", "region_id", "days_lead_time"]]

            merged = (
                candidates.merge(
                    week_supply,
                    how="left",
                    on="plant_id"
                )
                .query(
                    "model_code == @d_row.model_code and trim_code == @d_row.trim_code and color_code == @d_row.color_code"
                )
                .dropna(subset=["supply_qty"])
                .sort_values("days_lead_time")
            )

            for _, s_row in merged.iterrows():
                if remaining <= 0:
                    break
                available = s_row["supply_qty"]
                allocate = min(available, remaining)
                if allocate > 0:
                    allocations.append(
                        {
                            "scenario_id": scenario_id,
                            "week_start": week,
                            "plant_id": s_row["plant_id"],
                            "region_id": d_row["region_id"],
                            "model_code": d_row["model_code"],
                            "trim_code": d_row["trim_code"],
                            "color_code": d_row["color_code"],
                            "allocated_qty": int(allocate),
                            "implied_lead_days": int(s_row["days_lead_time"]),
                        }
                    )
                    week_supply.loc[
                        (week_supply["plant_id"] == s_row["plant_id"])
                        & (week_supply["model_code"] == s_row["model_code"])
                        & (week_supply["trim_code"] == s_row["trim_code"])
                        & (week_supply["color_code"] == s_row["color_code"]),
                        "supply_qty",
                    ] -= allocate
                    remaining -= allocate
    return pd.DataFrame(allocations)

In [46]:
#Baseline
alloc_baseline = run_greedy_allocation(base_supply, base_demand, lead_matrix, scenario_id=1)

In [47]:
#Fremont Incident
reduced_supply = base_supply.copy()
reduced_supply.loc[reduced_supply["plant_id"] == "FRE", "supply_qty"] *= 0.8
alloc_fremont_cut = run_greedy_allocation(reduced_supply, base_demand, lead_matrix, scenario_id=2)

/var/folders/jb/_hhv0vg53q5_6m25qfc8klvr0000gn/T/ipykernel_65909/614626507.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 57.6  15.2  34.4  61.6 159.2  35.2  88.8 128.8  36.   11.2  20.   47.2
 110.4  28.   62.4 106.4  71.2  28.8  41.6  62.4 160.8  47.2 101.6 164.
  54.4  12.8  20.   44.8 100.8  35.2  45.6 121.6  62.4  24.   43.2  68.
 178.4  48.8  89.6 176.   53.6  12.8  21.6  38.4  84.8  28.8  64.   88.
  68.8  20.   32.   71.2 157.6  52.8  86.4 165.6  44.   15.2  22.4  40.8
  96.   28.8  56.8 115.2  57.6  16.8  29.6  61.6 116.   31.2  84.8 148.8
  47.2   8.   20.8  54.4 103.2  27.2  49.6  93.6  64.8  16.8  44.   73.6
 145.6  40.   82.4 139.2  32.8  10.4  19.2  40.  102.4  36.8  60.8 113.6
  61.6  16.   25.6  68.  144.8  34.4  92.  135.2  35.2   4.8  22.4  45.6
  97.6  21.6  60.  103.2  56.   29.6  42.4  76.  149.6  48.8  78.4 146.4
  36.8  11.2  25.6  35.2 111.2  30.4  57.6  99.2  50.4  16.   27.2  54.4
 

In [ ]:
alloc_all = pd.concat([alloc_baseline, alloc_fremont_cut], ignore_index=True)
alloc_all.to_sql("allocation_result", con, if_exists="replace", index=False)
print(f"Allocation completed: {len(alloc_all)} total records written")

✅ Allocation completed: 1957 total records written


In [49]:
summary = (
    alloc_all.groupby(["scenario_id", "region_id"])["allocated_qty"]
    .sum()
    .reset_index()
    .pivot(index="region_id", columns="scenario_id", values="allocated_qty")
)
summary.columns = ["Baseline", "Fremont −20%"]
summary["Change %"] = (summary["Fremont −20%"] - summary["Baseline"]) / summary["Baseline"] * 100
print("\nRegional Allocation Comparison (% Change):\n", summary)


Regional Allocation Comparison (% Change):
            Baseline  Fremont −20%   Change %
region_id                                   
CC            32703         32057  -1.975354
EC            20036         18747  -6.433420
WC             4322          3874 -10.365571


In [50]:
alloc_all.to_csv("Allocation_result2.csv", index=False)